<a href="https://colab.research.google.com/github/MeGaNeKoS/Google_Colab/blob/master/GDrive_Video_converting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDrive Downloader 

In [ ]:
#@title Mount Google Drive
#@markdown To stream files we need to mount Google Drive.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# FFMPEG video converter

In [ ]:
#@title Install requirements
#@markdown To stream files we need to run ffmpeg.
!apt install ffmpeg
!sudo apt-get install mkvtoolnix-gui

### Convert video
Convert the video format using ffmpeg

In [ ]:
# @title Converting
# @markdown fill the parameter before converting. also check your notebook setting before using GPU

import os
from IPython.display import clear_output
import threading
import time

FOLDER_PATH = "/content/drive/Shared drives/Anime"  # @param {type:"string"}

# @markdown Video Format
IN_FORMAT = "mkv"  # @param {type:"string"}
OUT_FORMAT = "mkv"  # @param {type:"string"}
# @markdown Video codec
METHOD = "h.265"  # @param ["copy", "copy (-strict experimental)", "h.265"]
# @markdown Delete the source after convertion?
DELETE_SC = True  # @param {type:"boolean"}
# @markdown Include the sub folder?
SUB_FOLDER = True  # @param {type:"boolean"}
# @markdown Use GPU?
GPU = True  # @param {type:"boolean"}
# CLEAN_OUTPUT = False #@param {type:"boolean"}
BUFFER_SIZE = 1024

BEGIN = time.time()  # Set start time


def sizeof_fmt(num, suffix='B'):
    for unit in ['', 'K', 'M', 'G', 'T', 'P', 'E', 'Z']:
        if abs(num) < 1024.0:
            return f"{num:.3f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.3f}Yi{suffix}"


def duration(num, msg="ETA... ", sep=":"):
    if int((num - BEGIN) // 3600) > 0:
        msg += f"{int((num - BEGIN) // 3600)}{sep}"
    if int((num - BEGIN) // 60) > 0:
        msg += f"{int((num - BEGIN) // 60) % 60}{sep}"
    msg += f'{int((num - BEGIN) % 60)} sec'
    return " " + msg



def convert(src, param, out):
    global waiting
    !ffmpeg -hide_banner -loglevel panic -y -i "$src" $param "$out"
    waiting = False


files = []
fails = []
re_enc = []

ext = '.' + IN_FORMAT

# r=root, d=directories, f = files
if SUB_FOLDER:
    for r, d, f in os.walk(FOLDER_PATH, topdown=True):
        for file in f:
            if ext in file:
                files.append(os.path.join(r, file))
else:
    for name in os.listdir(FOLDER_PATH):
        if ext in name:
            files.append(name)

if IN_FORMAT == OUT_FORMAT:
    PATH_ALT = FOLDER_PATH + "_converted"
    DELETE_SC = False
    if not os.path.isdir(PATH_ALT):
        os.makedirs(PATH_ALT)
    if SUB_FOLDER:
        for r, d, f in os.walk(FOLDER_PATH, topdown=True):
            child = r.replace(FOLDER_PATH, PATH_ALT)
            if not os.path.isdir(child):
                os.mkdir(child)

files.sort()
MAX = len(files)
for idx, f in enumerate(files):
    if f.endswith(ext):
        if SUB_FOLDER:
            source = f
            output = f.replace("." + IN_FORMAT, "." + OUT_FORMAT)

        else:
            source = FOLDER_PATH + "/" + f
            output = FOLDER_PATH + "/" + f.replace("." + IN_FORMAT, "." + OUT_FORMAT)

        fixed_metadata = FOLDER_PATH + "_fixed" + output.replace(FOLDER_PATH, "")
        
        if IN_FORMAT == OUT_FORMAT:
            output = FOLDER_PATH + "_converted" + output.replace(FOLDER_PATH, "")

        os.makedirs(os.path.dirname(fixed_metadata), exist_ok=True)
        info = f"{f.split('/')[-1:][0]}\nConverted {idx} of {MAX}\n"
        time_remaining = f"{duration(time.time(), 'Time Elapsed')}"
        print(info + time_remaining, end='')
        title = len(info)
        print("\b" * len(time_remaining), end='')

        if METHOD == "copy":
            # !ffmpeg -hide_banner -loglevel panic -y -i "$source" -map 0:v -c:v copy -map 0:a -c:a aac -q:a 1 -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0 -max_muxing_queue_size $BUFFER_SIZE "$output"
            t1 = threading.Thread(target=convert, args=(source,
                                                        f'-map 0:v -c:v copy -map 0:a -c:a aac -q:a 1 -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0 -max_muxing_queue_size {BUFFER_SIZE}',
                                                        output), daemon=True)
            waiting = True
            t1.start()
            while waiting:
                time_remaining = f"{duration(time.time(), 'Time Elapsed')}"  # Duration
                print(f'\r{time_remaining}', end='')
                time.sleep(1)
            print("\b" * len(time_remaining), end='')

        elif METHOD == "copy (-strict experimental)":
            # !ffmpeg -hide_banner -loglevel panic -y -i "$source" -map 0:v -c:v copy -map 0:a -c:a copy -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0 -strict experimental -max_muxing_queue_size $BUFFER_SIZE "$output"
            t1 = threading.Thread(target=convert, args=(source,
                                                        f'-map 0:v -c:v copy -map 0:a -c:a aac -q:a 1 -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0 -strict experimental -max_muxing_queue_size {BUFFER_SIZE}',
                                                        output), daemon=True)
            waiting = True
            t1.start()
            while waiting:
                time_remaining = f"{duration(time.time(), 'Time Elapsed')}"  # Duration
                print(f'\r{time_remaining}', end='')
                time.sleep(1)
            print("\b" * len(time_remaining), end='')

        elif METHOD == "h.265":
            GPU_type = ""
            if GPU:
                ##Set up Variable
                Vx265 = f"Vx265.{OUT_FORMAT}"
                Vcopy = f"Vcopy.{OUT_FORMAT}"
                Aud = f"Audio.{OUT_FORMAT}"
                flag = True

                info = f"Converting the video using h.265\n"  # Status

                print(info, end='')

                # !ffmpeg -hide_banner -loglevel panic $GPU_type -y -i "$source" -map 0:v -c:v hevc_nvenc -preset hq -profile:v main10 -tier high -rc:v constqp -b:v 1M -maxrate:v 10M -rc-lookahead 40 -no-scenecut 1 -init_qpP 23 -init_qpB 25 -init_qpI 21 -weighted_pred 1 -max_muxing_queue_size $BUFFER_SIZE "$Vx265"
                t1 = threading.Thread(target=convert, args=(source,
                                                            f'-map 0:v -c:v hevc_nvenc -preset hq -profile:v main10 -tier high -rc:v constqp -maxrate:v 10M -init_qpP 23 -init_qpB 25 -init_qpI 21 -max_muxing_queue_size {BUFFER_SIZE}',
                                                            Vx265), daemon=True)
                waiting = True
                t1.start()
                while waiting:
                    time_remaining = f"{duration(time.time(), 'Time Elapsed')}"  # Duration
                    print(f'\r{time_remaining}', end='')
                    time.sleep(1)

                print("\b" * (len(info) + len(time_remaining)), end='')
                info = f"video h.265 converted\n"
                titVh = len(info)  # for delete the title video h.265 converted
                print(info, end='')

                info = f"Converting the video using copy\n"
                print(info, end='')

                # !ffmpeg -hide_banner -loglevel panic -y -i "$source" -map 0:v -c:v copy -map_chapters 0 -map_metadata 0 -max_muxing_queue_size $BUFFER_SIZE "$Vcopy"
                t1 = threading.Thread(target=convert, args=(
                source, f'-map 0:v -c:v copy -map_chapters 0 -map_metadata 0 -max_muxing_queue_size {BUFFER_SIZE}',
                Vcopy), daemon=False)
                waiting = True
                t1.start()
                while waiting:
                    time_remaining = f"{duration(time.time(), 'Time Elapsed')}"  # Duration
                    print(f'\r{time_remaining}', end='')
                    time.sleep(1)

                print("\b" * (len(info) + len(time_remaining)), end='')
                info = f"video copy converted\n"
                titVc = len(info)  # for delete the title video copy converted
                print(info, end='')

                # convert the video using copy
                info = f"Converting the audio\n"
                print(info, end='')

                # !ffmpeg -hide_banner -loglevel panic -y -i "$source" -map 0:a -c:a aac -q:a 1 -map_chapters 0 -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_metadata 0 -max_muxing_queue_size $BUFFER_SIZE "$Aud"
                !ffprobe -v error -select_streams a -show_entries stream=codec_name -of default=nokey=1:noprint_wrappers=1 "$source" > "temp.txt"
                with open("temp.txt", encoding="utf-8") as f:
                    audio_codecs = f.readlines()
                audio_codec = ""
                for index, codec in enumerate(audio_codecs):
                    if codec:
                        if codec.strip().lower() == "flac":
                            audio_codec += f" -map 0:a:{index} -c:a flac -compression_level 12"
                        else:
                            audio_codec += f" -map 0:a:{index} -c:a aac -q:a 0"
                t1 = threading.Thread(target=convert, args=(source,
                                                            f'{audio_codec} -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0 -max_muxing_queue_size {BUFFER_SIZE}',
                                                            Aud), daemon=True)
                waiting = True
                t1.start()
                while waiting:
                    time_remaining = f"{duration(time.time(), 'Time Elapsed')}"  # Duration
                    print(f'\r{time_remaining}', end='')
                    time.sleep(1)

                print("\b" * (len(info) + len(time_remaining)), end='')
                info = f"audio converted\n"
                titA = len(info)  # for delete the audio converted
                print(info, end='')

                info = f"Comparing video size :\n"
                time_remaining = f"{duration(time.time(), 'Time Elapsed')}"
                print(info + time_remaining, end='')
                titC = 0
                try:
                    x = os.stat(f"Vcopy.{OUT_FORMAT}").st_size
                    y = os.stat(f"Vx265.{OUT_FORMAT}").st_size
                    if x < 100 or y < 100:
                        fails.append(source)
                        flag = False
                except:
                    fails.append(source)
                    flag = False
                print("\b" * (len(info) + len(time_remaining)), end='')
                # compare the video
                if flag:
                    if x > y:
                        info = f"Combine Video (h.265) w/ audio\n"
                        time_remaining = f"{duration(time.time(), 'Time Elapsed')}"
                        print(info + time_remaining, end='')
                        !ffmpeg -hide_banner -loglevel panic -y -i "$Vx265" -i "$Aud" -map 1:a -map 0:v:0 -c:v copy -c:a copy -map 1:s:? -c:s copy -map 1:d:? -c:d copy -map 1:t:? -c:t copy -map_chapters 0 -map_metadata 0 -max_muxing_queue_size $BUFFER_SIZE "$output"
                    else:
                        re_enc.append(source)
                        info = f"Combine Video (copy) w/ audio"
                        time_remaining = f"\n{duration(time.time(), 'Time Elapsed')}"
                        print(info + time_remaining, end='')
                        !ffmpeg -hide_banner -loglevel panic -y -i "$Vcopy" -i "$Aud" -map 1:a -map 0:v:0 -c:v copy -c:a copy -map 1:s:? -c:s copy -map 1:d:? -c:d copy -map 1:t:? -c:t copy -map_chapters 0 -map_metadata 0 -max_muxing_queue_size $BUFFER_SIZE "$output"
                titC = len(info)
                print("\b" * len(time_remaining), end='')

                # output
                print("\b" * (titVh + titVc + titA + titC), end='')  # delete the convertion status
                try:
                    os.remove(Vx265)
                    os.remove(Vcopy)
                    os.remove(Aud)
                except:
                    pass

            else:
                # !ffmpeg -i "$source" -y -map 0:v -c:v libx265 -x265-params lossless -map 0:a -c:a aac -q:a 1 -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0 -max_muxing_queue_size $BUFFER_SIZE "$output"
                t1 = threading.Thread(target=convert, args=(source,
                                                            f'-y -map 0:v:0 -c:v libx265 -x265-params lossless -map 0:a -c:a aac -q:a 1 -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0 -max_muxing_queue_size {BUFFER_SIZE}',
                                                            output))
                while t1.isAlive():
                    time_remaining = f"{duration(time.time(), 'Time Elapsed')}"  # Duration
                    print(f'\r{time_remaining}', end='')
                    time.sleep(1)
                t1.join()
          
        try:
            x = os.stat(output).st_size

            if x > 0:
                if DELETE_SC:
                    os.remove(source)
                !mkvmerge -q -o "$fixed_metadata" "$output"
            else:
                if os.path.exists(output):
                    os.remove(output)
                    fails.append(source)
        except Exception:
            if os.path.exists(output):
                os.remove(output)
                fails.append(source)

        # if CLEAN_OUTPUT:
        #    clear_output()
        print("\b" * title, end='')

for f in re_enc:
    out = f.replace(FOLDER_PATH + '/', "")
    print(f"{out} is using copy to keep size small")
for f in fails:
    out = f.replace(FOLDER_PATH + '/', "")
    print(f"{out} failed to be converted")


def done():
    from google.colab import output
    output.eval_js(
        'new Audio("https://drive.google.com/uc?export=download&id=1h-KhKaO-FNMsRKKYtiZ-AqNy8tr5mHcX").play()')


print("done")
t1 = threading.Thread(target=done, args=(), daemon=True)
t1.start()
time.sleep(5)
if t1.is_alive():
    print(
        "\nYOU CAN IGNORE ANYTHING BELOW THIS.\nTHIS IS JUST TO FORCE STOP THE PROGRAM DUE AUDIO NOTIFICATION ERROR\n\n")
    raise SystemExit()  # sometime it got stuck when playing the notif audio.

In [ ]:
#@title Converting a single file
#@markdown Sometime error coz video format.

import os
from IPython.display import clear_output

SOURCE = ""#@param {type:"string"}
COMMAND = "-map 0:v -c:v hevc_nvenc -preset hq -profile:v main10 -tier high -rc:v constqp -b:v 1M -maxrate:v 10M -rc-lookahead 40 -no-scenecut 1 -init_qpP 23 -init_qpB 25 -init_qpI 21 -weighted_pred 1 -map 0:a -c:a aac -q:a 1 -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0"#@param {type:"string"}

#@markdown Video Format
IN_FORMAT = SOURCE[-3:]
OUT_FORMAT = "mp4" #@param {type:"string"}
#@markdown Delete the SOURCE after convertion?
DELETE_SC = True #@param {type:"boolean"}
GPU = "-hwaccel cuvid" #@param ["-hwaccel cuvid", "-hwaccel auto", ""]

files = []
fail = []
if IN_FORMAT == OUT_FORMAT:
    output = SOURCE.replace("." + IN_FORMAT, "-1." + OUT_FORMAT)
    DELETE_SC = False
else:
    output = SOURCE.replace("." + IN_FORMAT, "." + OUT_FORMAT)
!ffmpeg $GPU -i "$SOURCE" $COMMAND  "$output"

try :
    x = os.stat(output).st_size
    y = os.stat(SOURCE).st_size

    if x > (y + 5242880):
        os.remove(output)
        clear_output
        print("output file too big rather than using copy")
        !ffmpeg -i "$SOURCE" -map 0:a -map 0:v -c:v copy -c:a aac -q:a 1 -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0"$output"
    elif x > 0:
        if DELETE_SC:
            os.remove(SOURCE)
    else:
        if os.path.exists(output):
            os.remove(output)
        fail.append(SOURCE)
except:
    if os.path.exists(output):
        os.remove(output)
    print("failed to see file size")

for f in fail:
    out = f.replace(path+'/', "")
    print("%s failed to be converted" % (out))
print("done")
from google.colab import output
output.eval_js('new Audio("https://drive.google.com/uc?export=download&id=1h-KhKaO-FNMsRKKYtiZ-AqNy8tr5mHcX").play()')

### Tweek the video
Restore the files from the trash first

In [ ]:
#@title Reduce audio bitrate(subfolder)
#@markdown Dont overkill the audio
#@markdown using ffmpeg aac -q:a 1

import os
from IPython.display import clear_output
path = ""#@param {type:"string"}
files = []

fol = path + "/mp4"
if not os.path.exists(fol):
    os.makedirs(fol)
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.mp4' in file:
            files.append(os.path.join(r, file))

for f in files:
    source = f
    output = path +'/mp4'+f.replace(path, "")
    !ffmpeg -i "$source" -map 0:v -c:v copy -map 0:a -c:a aac -q:a 1 -map 0:s:? -c:s copy -map 0:d:? -c:d copy -map 0:t:? -c:t copy -map_chapters 0 -map_metadata 0"$output"
    clear_output()
print("done")
from google.colab import output
output.eval_js('new Audio("https://drive.google.com/uc?export=download&id=1h-KhKaO-FNMsRKKYtiZ-AqNy8tr5mHcX").play()')